# Single Bidding Zone with varying periods

In this example, we consider multiple time periods (labelled [1,2,3,4]) to represent variable wind generation and changing load.

  \begin{equation}
    \min_{g_{s,t}} \sum_s o_{s} g_{s,t}
  \end{equation}
  such that
  \begin{align}
    g_{s,t} &\leq \hat{g}_{s,t} G_{i,s} \\
    g_{s,t} &\geq 0 \\
    \sum_s g_{s,t} &= d_t
  \end{align}

In [1]:
import pyomo.environ as pe
import pandas as pd

In [2]:
wind_ts = [0.3, 0.6, 0.4, 0.5]
load_ts = [42000, 43000, 45000, 46000]

In [3]:
capacities = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000},
    "Mozambique": {"Hydro": 1200},
}

In [4]:
marginal_costs = {
    "Wind": 0,
    "Coal": 30,
    "Gas": 60,
    "Oil": 80,
    "Hydro": 0,
}

In [5]:
m = pe.ConcreteModel()
m.dual = pe.Suffix(direction = pe.Suffix.IMPORT)

In [6]:
techs = capacities['South Africa'].keys()

m.S = pe.Set(initialize = techs)
m.T = pe.RangeSet(4)

In [7]:
m.g = pe.Var(m.S, m.T, within = pe.NonNegativeReals)

In [8]:
m.cost = pe.Objective(expr = sum(marginal_costs[s] * m.g[s,t] for s in m.S for t in m.T))

In [9]:
@m.Constraint(m.S, m.T)
def generator_limit(m,s,t):
    cf = wind_ts[t-1] if s == "Wind" else 1
    return m.g[s,t] <= cf * capacities["South Africa"][s]

In [10]:
m.generator_limit.pprint()

generator_limit : Size=16, Index=S*T, Active=True
    Key         : Lower : Body      : Upper   : Active
    ('Coal', 1) :  -Inf : g[Coal,1] : 35000.0 :   True
    ('Coal', 2) :  -Inf : g[Coal,2] : 35000.0 :   True
    ('Coal', 3) :  -Inf : g[Coal,3] : 35000.0 :   True
    ('Coal', 4) :  -Inf : g[Coal,4] : 35000.0 :   True
     ('Gas', 1) :  -Inf :  g[Gas,1] :  8000.0 :   True
     ('Gas', 2) :  -Inf :  g[Gas,2] :  8000.0 :   True
     ('Gas', 3) :  -Inf :  g[Gas,3] :  8000.0 :   True
     ('Gas', 4) :  -Inf :  g[Gas,4] :  8000.0 :   True
     ('Oil', 1) :  -Inf :  g[Oil,1] :  2000.0 :   True
     ('Oil', 2) :  -Inf :  g[Oil,2] :  2000.0 :   True
     ('Oil', 3) :  -Inf :  g[Oil,3] :  2000.0 :   True
     ('Oil', 4) :  -Inf :  g[Oil,4] :  2000.0 :   True
    ('Wind', 1) :  -Inf : g[Wind,1] :   900.0 :   True
    ('Wind', 2) :  -Inf : g[Wind,2] :  1800.0 :   True
    ('Wind', 3) :  -Inf : g[Wind,3] :  1200.0 :   True
    ('Wind', 4) :  -Inf : g[Wind,4] :  1500.0 :   True


In [11]:
@m.Constraint(m.T)
def energy_balance(m, t):
    return sum(m.g[s,t] for s in m.S) == load_ts[t-1]

In [12]:
m.energy_balance.pprint()

energy_balance : Size=4, Index=T, Active=True
    Key : Lower   : Body                                        : Upper   : Active
      1 : 42000.0 : g[Coal,1] + g[Wind,1] + g[Gas,1] + g[Oil,1] : 42000.0 :   True
      2 : 43000.0 : g[Coal,2] + g[Wind,2] + g[Gas,2] + g[Oil,2] : 43000.0 :   True
      3 : 45000.0 : g[Coal,3] + g[Wind,3] + g[Gas,3] + g[Oil,3] : 45000.0 :   True
      4 : 46000.0 : g[Coal,4] + g[Wind,4] + g[Gas,4] + g[Oil,4] : 46000.0 :   True


In [13]:
%%timeit -n 1 -r 1
pe.SolverFactory("gurobi").solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 6082000.0
  Upper bound: 6082000.0
  Number of objectives: 1
  Number of constraints: 20
  Number of variables: 16
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 16
  Number of nonzeros: 32
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality

In [14]:
m.cost()

6082000.0

In [15]:
pd.Series(m.dual.values(), m.dual.keys())

generator_limit[Coal,1]   -30.0
generator_limit[Coal,2]   -30.0
generator_limit[Coal,3]   -50.0
generator_limit[Coal,4]   -50.0
generator_limit[Wind,1]   -60.0
generator_limit[Wind,2]   -60.0
generator_limit[Wind,3]   -80.0
generator_limit[Wind,4]   -80.0
generator_limit[Gas,1]      0.0
generator_limit[Gas,2]      0.0
generator_limit[Gas,3]    -20.0
generator_limit[Gas,4]    -20.0
generator_limit[Oil,1]      0.0
generator_limit[Oil,2]      0.0
generator_limit[Oil,3]      0.0
generator_limit[Oil,4]      0.0
energy_balance[1]          60.0
energy_balance[2]          60.0
energy_balance[3]          80.0
energy_balance[4]          80.0
dtype: float64

In [16]:
pd.Series(m.g.get_values()).unstack(0)

,Coal,Gas,Oil,Wind
1,35000.0,6100.0,0.0,900.0
2,35000.0,6200.0,0.0,1800.0
3,35000.0,8000.0,800.0,1200.0
4,35000.0,8000.0,1500.0,1500.0


# Adding Energy Storage

In [17]:
storage_energy = 6000  # MWh
storage_power = 1000  # MW
efficiency = 0.9  # discharge = charge
standing_loss = 0.00001  # per hour

In [18]:
m.discharge = pe.Var(m.T, bounds = (0, storage_power))
m.charge = pe.Var(m.T, bounds = (0, storage_power))
m.soc = pe.Var(m.T, bounds = (0, storage_energy))

In [19]:
@m.Constraint(m.T)
def storage_consistency(m,t):
    if t == 1:
        return m.soc[t] == 0
    return (m.soc[t] == (1 - standing_loss) * m.soc[t-1] + efficiency * m.charge[t] - 1/efficiency * m.discharge[t])

In [20]:
m.storage_consistency.pprint()

storage_consistency : Size=4, Index=T, Active=True
    Key : Lower : Body                                                                        : Upper : Active
      1 :   0.0 :                                                                      soc[1] :   0.0 :   True
      2 :   0.0 : soc[2] - (0.99999*soc[1] + 0.9*charge[2] - 1.1111111111111112*discharge[2]) :   0.0 :   True
      3 :   0.0 : soc[3] - (0.99999*soc[2] + 0.9*charge[3] - 1.1111111111111112*discharge[3]) :   0.0 :   True
      4 :   0.0 : soc[4] - (0.99999*soc[3] + 0.9*charge[4] - 1.1111111111111112*discharge[4]) :   0.0 :   True


In [21]:
m.del_component(m.energy_balance)

In [22]:
@m.Constraint(m.T)
def energy_balance (m,t):
    return sum(m.g[s,t] for s in m.S) + m.discharge[t] - m.charge[t] == load_ts[t-1]

In [23]:
pe.SolverFactory("gurobi").solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 6017200.65599352
  Upper bound: 6017200.65599352
  Number of objectives: 1
  Number of constraints: 24
  Number of variables: 28
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 28
  Number of nonzeros: 53
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved

In [24]:
m.cost()

6017200.65599352

In [25]:
pd.Series(m.dual.values(), m.dual.keys())

generator_limit[Coal,1]   -30.00000
generator_limit[Coal,2]   -30.00000
generator_limit[Coal,3]   -49.99920
generator_limit[Coal,4]   -50.00000
generator_limit[Wind,1]   -60.00000
generator_limit[Wind,2]   -60.00000
generator_limit[Wind,3]   -79.99920
generator_limit[Wind,4]   -80.00000
generator_limit[Gas,1]      0.00000
generator_limit[Gas,2]      0.00000
generator_limit[Gas,3]    -19.99920
generator_limit[Gas,4]    -20.00000
generator_limit[Oil,1]      0.00000
generator_limit[Oil,2]      0.00000
generator_limit[Oil,3]      0.00000
generator_limit[Oil,4]      0.00000
storage_consistency[1]    -71.99784
storage_consistency[2]    -71.99856
storage_consistency[3]    -71.99928
storage_consistency[4]    -72.00000
energy_balance[1]          60.00000
energy_balance[2]          60.00000
energy_balance[3]          79.99920
energy_balance[4]          80.00000
dtype: float64

In [26]:
pd.Series(m.discharge.get_values())

1    1000.0000
2       0.0000
3     800.0000
4       9.9918
dtype: float64

In [27]:
pd.Series(m.charge.get_values())

1       0.0
2    1000.0
3       0.0
4       0.0
dtype: float64

In [28]:
pd.Series(m.soc.get_values())

1      0.000000
2    900.000000
3     11.102111
4      0.000000
dtype: float64